## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [845]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
%matplotlib qt

In [858]:
def get_camera_calibration(path='camera_cal/calibration*.jpg', show=False, save_path=None):

    # prepare object points
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob(path)


    # Step through the list and search for chessboard corners
    for i, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)


        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)

            if save_path is not None:
                cv2.imwrite(save_path + 'chess_corners{}.jpg'.format(i),img)

            if show is True:
                cv2.imshow('img',img)
                cv2.waitKey(500)
    
    if show is True:
        cv2.destroyAllWindows()
        
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
        
    return mtx, dist

In [859]:
mtx, dist = get_camera_calibration(show=False, save_path='output_images/')

In [860]:
def undistort_img(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

In [42]:
def udistort_images(mtx, dist, path='camera_cal/calibration*.jpg', save_path=None):
    images = glob.glob(path)
        
    for i, fname in enumerate(images):
        img = cv2.imread(fname)
        img = undistort_img(img, mtx, dist)
        if save_path is not None:
            cv2.imwrite(save_path + 'test_undist{}.jpg'.format(i),img)

In [43]:
udistort_images(mtx, dist, path='test_images/*.jpg', save_path='test_images_undist/')

In [4]:
def save_as(fname, dest_path, suffix):
    # remove leading /
    fname = fname.split('/')
    # add suffix. before the .
    fname = fname[-1].replace('.', '_' + suffix + '.')
    
    return dest_path + fname  

In [45]:
#%matplotlib inline

In [235]:
def region_of_interest(img):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    
    h,w = img.shape[0], img.shape[1]
        
    vertices = np.array([[[int(0.15*w),int(.95*h)], # bot left
                          [int(.45*w),int(.58*h)],  # top left
                          [int(.55*w),int(.58*h)],  # top right
                          [int(.9*w),int(.95*h)]]],# bot right
                            dtype=np.int32 )
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [286]:
# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 255

    # Return the binary image
    return binary_output

In [287]:
# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh1(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 255

    # Return the binary image
    return binary_output

In [288]:
# sobel
# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 255

    # Return the result
    return binary_output

In [318]:
# Define a function that thresholds the S-channel of HLS
def hls_select(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 255
    #return binary_output
    #print(binary_output)
    return binary_output
    #return s_channel

In [342]:
def try_threshold(path):
    images = glob.glob(path)
    
    #plt.subplot(len(images)//2, 2)
    #f= plt.subplots(len(images)//2, 2, figsize=(24, 9))
    #f.tight_layout()
    plt.figure(figsize=(8,10), tight_layout=True)
    plt.axis('off')
    
    for i, fname in enumerate(images):
        img = cv2.imread(fname)
        #out = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        s_binary = hls_select(img, thresh=(180,255))
        sxbinary = abs_sobel_thresh(img, orient='x', thresh_min=30, thresh_max=100)
        #sxbinary = dir_threshold(img, thresh=(45./180.*np.pi, 60./180.*np.pi))
        
        combined_binary = np.zeros_like(s_binary)
        combined_binary[(s_binary == 255) | (sxbinary == 255)] = 255
            
        color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
        
        #out = region_of_interest(color_binary)
        out = region_of_interest(combined_binary)
        
        #out = dir_threshold(img, thresh=(45./180.*np.pi, 60./180.*np.pi))
        #out = mag_thresh1(img, mag_thresh=(15,100))
        #cv2.imshow('img'+str(i),out)
        
        ax = plt.subplot(len(images)//2, 2, i+1)
        ax.axis('off')
        plt.imshow(out, cmap='gray')
        #plt.imshow(out)
        #cv2.waitKey(1000)
    
    #print(img.shape)
    #cv2.destroyAllWindows()

In [343]:
try_threshold('test_images_undist/*.jpg')

In [561]:
def get_perspective_transform(img_shape):
    
    h,w = img.shape[0], img.shape[1]
    
    top_most = 460
    offset = 50
    
    src = np.float32(
        [[579, top_most],
         [203, h],
         [1127, h],
         [704, top_most]])

    dst = np.float32(
        [[320+offset, 0],
         [320+offset, h],
         [960-offset, h],
         [960-offset, 0]])
    
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    return M, Minv

In [822]:
M, Minv = get_perspective_transform((720,1280))

In [551]:
def warp(img, M):
    img_size = (img.shape[1], img.shape[0])
    return cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

In [518]:
img = cv2.imread('test_images_undist/test_undist0.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#img = region_of_interest(img)
plt.figure(figsize=(8,10), tight_layout=True)
plt.axis('off')
#plt.imshow(img)
plt.subplot(1, 2, 1)
plt.imshow(img)
#plt.plot(src,'r.')
plt.subplot(1, 2, 2)
warped = warp(img, M)
plt.imshow(warped)

In [563]:
def show_transform(path):
    images = glob.glob(path)
    
    plt.figure(figsize=(8,10), tight_layout=True)
    plt.axis('off')
    
    for i, fname in enumerate(images):
        img = cv2.imread(fname)
        out = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        out = warp(out, M)
        ax = plt.subplot(len(images)//2, 2, i+1)
        ax.axis('off')
        plt.imshow(out)

In [564]:
show_transform('test_images_undist/*.jpg')

In [612]:
def pipeline(fname):
    img = cv2.imread(fname)
    s_binary = hls_select(img, thresh=(170,255)) #175,255
    sxbinary = abs_sobel_thresh(img, orient='x', thresh_min=20, thresh_max=150) #30/100
    combined_binary = np.zeros_like(s_binary)
    combined_binary[(s_binary == 255) | (sxbinary == 255)] = 255
    out = region_of_interest(combined_binary)
    out = warp(out, M)
    return out

In [849]:
def color_and_grandient_threshold(img):
    s_binary = hls_select(img, thresh=(170,255))
    sxbinary = abs_sobel_thresh(img, orient='x', thresh_min=20, thresh_max=150)
    combined_binary = np.zeros_like(s_binary)
    combined_binary[(s_binary == 255) | (sxbinary == 255)] = 255
    out = region_of_interest(combined_binary)
    #out = warp(out, M)
    
    return out 

In [846]:
def show_pipeline(path):
    images = glob.glob(path)
    
    plt.figure(figsize=(8,10), tight_layout=True)
    plt.axis('off')
    
    for i, fname in enumerate(images):
        out = pipeline(fname)
        plt.subplot(len(images)//2, 2, i+1)
        plt.imshow(out, cmap='gray')

In [847]:
show_pipeline('test_images_undist/*.jpg')

In [831]:
img = pipeline('test_images_undist/test_undist6.jpg')
out_img = np.dstack((img, img, img))*255

In [832]:
plt.figure(figsize=(8,10), tight_layout=True)
#histogram = np.sum(img[int(img.shape[0]/2):,:], axis=0)
#plt.plot(histogram)
#plt.imshow(img, cmap='gray')
plt.imshow(img)

In [782]:
def find_baseline_x(img, lane='left'):
    # histogram for lower half
    histogram = np.sum(img[np.int(img.shape[0]/2):,:], axis=0)
    
    # find midpoint along x
    midpoint = np.int(histogram.shape[0]/2)
    
    if lane is 'left':
        x_base = np.argmax(histogram[:midpoint])
    else:
        x_base = np.argmax(histogram[midpoint:]) + midpoint
    
    return x_base

In [802]:
def roc_and_loc(x, y):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    y_eval_m = 720*ym_per_pix
    x = x*xm_per_pix
    y = y*ym_per_pix
    
    fit = np.polyfit(y, x, 2)
    
    r = ((1 + (2.*fit[0]*y_eval_m + fit[1])**2)**1.5) / np.absolute(2*fit[0])
    
    # evaluate fit at the bottom of image
    loc = fit[0]*y_eval_m**2 + fit[1]*y_eval_m + fit[2]
    
    center_of_image_m = (1280. / 2.) * xm_per_pix
    
    loc -= center_of_image_m
    
    return r, loc

In [833]:
left_x_base = find_baseline_x(img)

In [834]:
right_x_base = find_baseline_x(img, lane='right')

In [835]:
def sliding_window_search(img, x_base, debug=False, out_img=None):
    
    # set parameters
    # number of sliding windows (height dir)
    nwindows = 9
    
    # width of the sliding search window is +/- margin
    margin = 100
    # recenter next window if at least this many points were found
    minpix = 50
    
    #if debug is True:
    #    out_img = np.dstack((img, img, img))*255
    
    # Set height of windows
    window_height = np.int(img.shape[0]/nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated for each window
    x_current = x_base
    
    # Create empty lists to receive left and right lane pixel indices
    lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        
        win_x_low = x_current - margin
        win_x_high = x_current + margin
        
        if debug is True and out_img is not None:
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_x_low,win_y_low),(win_x_high,win_y_high),(0,255,0), 2)
            
        
        # Identify the nonzero pixels in x and y within the window
        good_inds = ((nonzeroy >= win_y_low) &
                     (nonzeroy < win_y_high) & 
                     (nonzerox >= win_x_low) & 
                     (nonzerox < win_x_high)).nonzero()[0]
        
        # Append these indices to the lists
        lane_inds.append(good_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_inds) > minpix:
            x_current = np.int(np.mean(nonzerox[good_inds]))


    # Concatenate the arrays of indices
    lane_inds = np.concatenate(lane_inds)

    # Extract left and right line pixel positions
    x = nonzerox[lane_inds]
    y = nonzeroy[lane_inds] 
    
    if debug is True and out_img is not None:
        out_img[nonzeroy[lane_inds], nonzerox[lane_inds]] = [255, 0, 0]


    # Fit a second order polynomial
    fit = np.polyfit(y, x, 2)
    
    # find ROC and lane location at bottom of image
    r, loc = roc_and_loc(x, y)
    
    #if debug is True:
    #    plt.figure(figsize=(8,10), tight_layout=True)
    #    plt.imshow(out_img)

    return fit, r, loc, out_img

In [836]:
fit_left, r_left, loc_left, out_img = sliding_window_search(img, left_x_base, debug=True, out_img=out_img)
print(r_left)
print(loc_left)

487.674940374
-1.43133753413


In [837]:
fit_right, r_right, loc_right, out_img = sliding_window_search(img, right_x_base, debug=True, out_img=out_img)
print(r_right)
print(loc_right)

1161.85361622
1.54026468265


In [838]:
print(loc_left + loc_right)

0.108927148519


In [789]:
def visualize_fit(img, left_fit, right_fit, out_img, single_image=False):
    # Generate x and y values for plotting
    
    if single_image:
        plt.figure(figsize=(8,10), tight_layout=True)
    
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)

In [790]:
visualize_fit(img, fit_left, fit_right, out_img, single_image=True)

In [746]:
def visualize_fit_batch(path):
    images = glob.glob(path)
    
    plt.figure(figsize=(8,10), tight_layout=True)
    plt.axis('off')
    
    for i, fname in enumerate(images):
        img = pipeline(fname)
        out_img = np.dstack((img, img, img))*255
        
        plt.subplot(len(images)//2, 2, i+1)
        
        left_x_base = find_baseline_x(img, lane='left')
        right_x_base = find_baseline_x(img, lane='right')
        
        fit_left, out_img = sliding_window_search(img, left_x_base, debug=True, out_img=out_img)
        fit_right, out_img = sliding_window_search(img, right_x_base, debug=True, out_img=out_img)
        
        visualize_fit(img, fit_left, fit_right, out_img)

In [747]:
visualize_fit_batch('test_images_undist/*.jpg')

In [757]:
def local_fit_search(img, fit, debug=False, out_img=None):
    # search based on fit from pervious images
    
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    margin = 100
    
    lane_inds = ((nonzerox > (fit[0]*(nonzeroy**2) + fit[1]*nonzeroy + fit[2] - margin)) &
                 (nonzerox < (fit[0]*(nonzeroy**2) + fit[1]*nonzeroy + fit[2] + margin))) 

    # Extract line pixel positions
    x = nonzerox[lane_inds]
    y = nonzeroy[lane_inds] 
    
    # Fit a second order polynomial
    fit = np.polyfit(y, x, 2)
    
    if debug is True and out_img is not None:
        out_img[nonzeroy[lane_inds], nonzerox[lane_inds]] = [255, 0, 0]

    return fit, out_img

In [779]:
out_img = np.dstack((img, img, img))*255
fit_left, out_img = local_fit_search(img, fit_left, debug=True, out_img=out_img)
fit_right, out_img = local_fit_search(img, fit_right, debug=True, out_img=out_img)
visualize_fit(img, fit_left, fit_right, out_img, single_image=True)

In [936]:
def draw_overlay(img, fit_l, fit_r, Minv, r_left, r_right, loc_error):
    
    # Create an image to draw the lines on
    color_warp = np.zeros_like(img).astype(np.uint8)
    #color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
    left_fitx = fit_l[0]*ploty**2 + fit_l[1]*ploty + fit_l[2]
    right_fitx = fit_r[0]*ploty**2 + fit_r[1]*ploty + fit_r[2]

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(result, '{:s}: {:>+5.2f} m'.format('lane centering', loc_error), (450,700), font, 1,(255,255,255),2)
    cv2.putText(result, '{:s}: {:+.2f} m'.format('ROC', r_left), (50,700), font, 1,(255,255,255),2)
    cv2.putText(result, '{:s}: {:+.2f} m'.format('ROC', r_right), (970,700), font, 1,(255,255,255),2)
    
    return result

In [938]:
plt.figure(figsize=(8,10), tight_layout=True)
plt.axis('off')
img = cv2.imread('test_images_undist/test_undist6.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
ax = plt.subplot(1, 2, 1)
ax.axis('off')
plt.imshow(img)

In [939]:
loc_error = loc_left + loc_right
result = draw_overlay(img, fit_left, fit_right, Minv, r_left, r_right, loc_error)
result = process_image(img)
ax = plt.subplot(1, 2, 2)
ax.axis('off')
plt.imshow(result)

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

In [885]:
def process_image(img):
    # distortion correction
    img = undistort_img(img, mtx, dist)
    
    # color and gradient thresholding
    img_th = color_and_grandient_threshold(img)
    
    # perspective transform
    img_th = warp(img_th, M)
    
    # line identification
    left_x_base = find_baseline_x(img_th, lane='left')
    right_x_base = find_baseline_x(img_th, lane='right')
    
    fit_left, r_left, loc_left, _ = sliding_window_search(img_th, left_x_base)
    fit_right, r_right, loc_right, _ = sliding_window_search(img_th, right_x_base)
    
    loc_error = loc_left + loc_right
    
    # line drawing and perspetive inverse transform
    out = draw_overlay(img, fit_left, fit_right, Minv, r_left, r_right, loc_error)
    
    return out

In [862]:
output_video = 'output_video.mp4'
clip = VideoFileClip("project_video.mp4")
out_clip = clip.fl_image(process_image) #NOTE: this function expects color images!!
out_clip.write_videofile(output_video, audio=False)

[MoviePy] >>>> Building video output_video.mp4
[MoviePy] Writing video output_video.mp4


100%|█████████▉| 1260/1261 [02:59<00:00,  6.94it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_video.mp4 

